## A stateless oracle (4): the Smart Signatures
#### 09.4 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-02-15
* Part 4: Create the accounts and Smart Signatures that make it possible to use the Oracle
* Parts 1-4 are only relevant if you want to **create** an Oracle
* Only parts 5-6 are needed to **use** the oracle.

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [2]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# additional oracle accounts
cred_oracle = load_credentials('credentials_oracle')
Price = cred_oracle['Price']
Reserve = cred_oracle['Reserve']
oracle_id = cred_oracle['oracle_id']

In [22]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction, LogicSigAccount
from algosdk.future.transaction import PaymentTxn

import algosdk.error
import json
import base64
import hashlib

In [23]:
from pyteal import *

In [24]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [25]:
print(Price['public'])
print(Reserve['public'])

A3ZHXQJQ6UF4WST5YGTZ2X6VSBGO5MQINJYJSEVY3Y7QSRQVGDCSQSRQOE
JKHCUTKXNEP4ERPZZW6SCRHZXUDI7NY34C45XVF7NLFHAU3C55P7Q6YPZU


In [5]:
import json
import requests
import pandas as pd
import numpy as np
import time

## Get information about the oracle coin

In [6]:
print('https://testnet.algoexplorer.io/asset/{}'.format(oracle_id))

https://testnet.algoexplorer.io/asset/77534697


## Create Smart Signatures
* We create delegated Smart Signatures for `Price` and `Reserve`
* The main condition is the `self_cond`, namely that the smart signature will only authorize transactions to itself
* The rest are safety conditions

### Delegated Smart Signature for Price
* This code generates the Delegated Smart Signature for Price and for Reserve
* It is for educational purposes, in the end we will use the version with the for-loop in the appendix

In [75]:
# Chage this to Reserve if need be
account = Price

In [76]:
# Step 1: Prepare PyTeal
self_cond = And (
    Txn.asset_sender() == Addr(account['public']),     # Send from Price
    Txn.asset_receiver() == Addr(account['public']),   # ... to self
    Txn.xfer_asset() == Int(oracle_id)               # Oracle Coin ID
)

fee_cond = Txn.fee() <= Int(1000)

safety_cond = And(
    Txn.type_enum() == TxnType.AssetTransfer,             # Must be an "asset transfer" transaction
    Txn.close_remainder_to() == Global.zero_address(),
    Txn.rekey_to() == Global.zero_address(),
    )
    
delegatedSig_pyteal = And(
    self_cond,
    fee_cond, 
    safety_cond
    )

In [77]:
# Step 2: Pyteal -> Compile Teal
delegatedSig_teal = compileTeal(delegatedSig_pyteal, Mode.Signature, version=3)
print(delegatedSig_teal)

#pragma version 3
txn AssetSender
addr A3ZHXQJQ6UF4WST5YGTZ2X6VSBGO5MQINJYJSEVY3Y7QSRQVGDCSQSRQOE
==
txn AssetReceiver
addr A3ZHXQJQ6UF4WST5YGTZ2X6VSBGO5MQINJYJSEVY3Y7QSRQVGDCSQSRQOE
==
&&
txn XferAsset
int 77534697
==
&&
txn Fee
int 1000
<=
&&
txn TypeEnum
int axfer
==
txn CloseRemainderTo
global ZeroAddress
==
&&
txn RekeyTo
global ZeroAddress
==
&&
&&
return


In [78]:
# Step 3: Compile Teal -> Bytecode for AVM
delegatedSig_compiled = algod_client.compile(delegatedSig_teal)
delegatedSig_encoded_prog = delegatedSig_compiled['result'].encode()
delegatedSig_program = base64.decodebytes(delegatedSig_encoded_prog)
lsig = LogicSigAccount(delegatedSig_program)

In [79]:
# Step 4: Price account signs Smart Signature
lsig.sign(account['private'])

#### The owner of the oracle communicates the following pieces of information

In [80]:
# (1) The program logic (the rules)
print(delegatedSig_encoded_prog)

# (2) The signature of the program
delegatedSig_sig = lsigPrice.lsig.sig
print(delegatedSig_sig)

# (3) The signing account (=Price['public'])
delegatedSig_acc = base64.b32encode(lsigPrice.sigkey)
print(delegatedSig_acc)

b'AyAD6av8JOgHBCYBIAbye8Ew9QvLSn3Bp51f1ZBM7rIIanCZErjePwlGFTDFMRMoEjEUKBIQMREiEhAxASMOEDEQJBIxCTIDEhAxIDIDEhAQQw=='
zgx57EMpZlHp8YJ7Ntol1uZFzQSkvj+JQHQZn5srB/pjFu3JQEMjq4IqcTw1nA+YAkYoFDpcrEvQI6HeyJn/CQ==
b'A3ZHXQJQ6UF4WST5YGTZ2X6VSBGO5MQINJYJSEVY3Y7QSRQVGDCQ===='


## Appendix: A more efficient version of the code
* In a real-world setting, we would not copy-paste the code
* We would use functions and loops

**Note** there is no harm in running this code in addition to the "simple" version, as we do not create any blockchain transactions

In [177]:
# Function that creates the Smart Signatures
def create_smart_sig(account):
    # Step 1: Prepare PyTeal
    self_cond = And (
        Txn.sender() == Addr(account['public']),           # Send from Price 
        Txn.asset_receiver() == Addr(account['public']),   # ... to self
        Txn.xfer_asset() == Int(oracle_id)                 # Oracle Coin ID
    )

    fee_cond = Txn.fee() <= Int(1000)

    safety_cond = And(
        Txn.type_enum() == TxnType.AssetTransfer,             # Must be an "asset transfer" transaction
        Txn.close_remainder_to() == Global.zero_address(),
        Txn.rekey_to() == Global.zero_address(),
        )
    
    delegatedSig_pyteal = And(
        self_cond,
        fee_cond, 
        safety_cond
        )

    # Step 2: Pyteal -> Compile Teal
    delegatedSig_teal = compileTeal(delegatedSig_pyteal, Mode.Signature, version=3)
    #print(delegatedSig_teal)

    # Step 3: Compile Teal -> Bytecode for AVM
    delegatedSig_compiled = algod_client.compile(delegatedSig_teal)
    delegatedSig_encoded_prog = delegatedSig_compiled['result'].encode()
    delegatedSig_program = base64.decodebytes(delegatedSig_encoded_prog)
    lsig = LogicSigAccount(delegatedSig_program)#

    # Step 4: Price account signs Smart Signature
    lsig.sign(account['private'])

    smartSig = {}
    smartSig['program'] = delegatedSig_encoded_prog
    smartSig['sig']     = lsig.lsig.sig    # delegatedSig_sig = 
    smartSig['public']  =  base64.b32encode(lsig.sigkey)   # delegatedSig_acc
    smartSig['public_addr']  = account['public']
    return(smartSig)

In [191]:
print('Price')
price_sig = create_smart_sig(Price)
print ( create_smart_sig(Price) )
print('Reserve')
reserve_sig = create_smart_sig(Price)
print ( reserve_sig )

Price
{'program': b'AyAD6av8JOgHBCYBIAbye8Ew9QvLSn3Bp51f1ZBM7rIIanCZErjePwlGFTDFMQAoEjEUKBIQMREiEhAxASMOEDEQJBIxCTIDEhAxIDIDEhAQQw==', 'sig': 'l87eZLMhhdkpPHos+B/eZJC/f01NoHBy2jW4B3ofYMalN9e/lyszBGdLt7h0XH0tECqTegG1wJasZGGQa2LCCg==', 'public': b'A3ZHXQJQ6UF4WST5YGTZ2X6VSBGO5MQINJYJSEVY3Y7QSRQVGDCQ====', 'public_addr': 'A3ZHXQJQ6UF4WST5YGTZ2X6VSBGO5MQINJYJSEVY3Y7QSRQVGDCSQSRQOE'}
Reserve
{'program': b'AyAD6av8JOgHBCYBIAbye8Ew9QvLSn3Bp51f1ZBM7rIIanCZErjePwlGFTDFMQAoEjEUKBIQMREiEhAxASMOEDEQJBIxCTIDEhAxIDIDEhAQQw==', 'sig': 'l87eZLMhhdkpPHos+B/eZJC/f01NoHBy2jW4B3ofYMalN9e/lyszBGdLt7h0XH0tECqTegG1wJasZGGQa2LCCg==', 'public': b'A3ZHXQJQ6UF4WST5YGTZ2X6VSBGO5MQINJYJSEVY3Y7QSRQVGDCQ====', 'public_addr': 'A3ZHXQJQ6UF4WST5YGTZ2X6VSBGO5MQINJYJSEVY3Y7QSRQVGDCSQSRQOE'}


## Appendix: Test the smart signature(s)

In [198]:
# choose price_sig or reserve_sig
my_sig = price_sig

In [196]:
# Step 0.1: create a new smart signature object from the program logic  --> see WSC 06.6
lsigTest = LogicSigAccount(base64.decodebytes(my_sig['program']))
# Step 0.2: add the signature
lsigTest.lsig.sig = my_sig['sig']
# Step 0.3: add the public key of the siging account (Price)
lsigTest.sigkey = base64.b32decode(my_sig['public'])

In [202]:
# Step 1: prepare Txn
amt = int(1e5)                                   # for testing, any amount is good
sender = lsigTest.address()
receiver = lsigTest.address()

sp = algod_client.suggested_params()          
txn = AssetTransferTxn(sender=sender, sp=sp, receiver=receiver, amt=amt, index=oracle_id)

# Step 2: Smart Sig signs txn
lstx = transaction.LogicSigTransaction(txn, lsigTest)   # <--------- Signature recreated by Bob 

# Step 3: Send
txid = algod_client.send_transaction(lstx)

# Step 4: Wait for confirmation
confirmed_txn = wait_for_confirmation(algod_client, txid)

Current round is  20483067.
Waiting for round 20483067 to finish.
Waiting for round 20483068 to finish.
Transaction UTRHFGM5EIXNQAVAVZEI2KF5TTYNGBWNS5R2FARDPTZ5CSCTKEHA confirmed in round 20483069.
